In [1]:
import sys
from pathlib import Path

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
#settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /Users/isabell/Documents/KTH/KTH femman/Scalable Machine learning/id2223-lab1


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 🚨 Select running mode

In [2]:
#Select whether part E, C or A is run

running_e = False
running_c = False
running_a = True 

### <span style='color:#ff5f27'> 📝 Imports

In [3]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [4]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value

if running_e or running_c:
    location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
    location = json.loads(location_str)

    country=location['country']
    city=location['city']
    street=location['street']
    aqicn_url=location['aqicn_url']
    latitude=location['latitude']
    longitude=location['longitude']

if running_a:
    location_str_array = secrets.get_secret("SENSOR_LOCATIONS_JSON").value #Note different secret, the one with all Edinburgh locations
    location_array = json.loads(location_str_array)
    print(location_array)

    country=location_array[0]['country']
    city=location_array[0]['city']
   
    latitude=location_array[0]['latitude']
    longitude=location_array[0]['longitude']

    location_str_array



today = datetime.date.today()



2025-11-18 18:30:59,285 INFO: Initializing external client
2025-11-18 18:30:59,286 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-18 18:31:00,923 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1272015
[{'country': 'united-kingdom', 'city': 'edinburgh', 'street': 'tower street', 'aqicn_url': 'https://api.waqi.info/feed/@11656/', 'latitude': '55.956258', 'longitude': '-3.191111'}, {'country': 'united-kingdom', 'city': 'edinburgh', 'street': 'salamander st', 'aqicn_url': 'https://api.waqi.info/feed/@5986/', 'latitude': '55.956258', 'longitude': '-3.191111'}, {'country': 'united-kingdom', 'city': 'edinburgh', 'street': 'st leonards', 'aqicn_url': 'https://api.waqi.info/feed/@5987/', 'latitude': '55.956258', 'longitude': '-3.191111'}, {'country': 'united-kingdom', 'city': 'edinburgh', 'street': 'balmwell terrace', 'aqicn_url': 'https://api.waqi.info/feed/A93199/', 'latitude': '55.956258', 'longitude': '-3.191111'}, {'country': 'united-kingdom', 'city': 'edinburgh', 'street': "st john's road", 'aqicn_url': 'https://api.waqi.info/feed/@9580/', 'latitude':

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [5]:
#Select version of feature group
if running_e:
    aq_version = 1
    w_version = 1

if running_c:
    aq_version = 2
    w_version = 1

if running_a:
    aq_version = 3
    w_version = 2

# Retrieve feature groups

air_quality_fg = fs.get_feature_group( 
    name='air_quality',
    version=aq_version,
)

weather_fg = fs.get_feature_group(
    name='weather',
    version=w_version,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [6]:
import requests
import pandas as pd

if running_e or running_c:
    aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)


#If running A data must be retrueved from each sensor
if running_a:
    aq_today_df = None   # TA INTE BORT! 


    for location in location_array:
        url = location["aqicn_url"]
        addr = location["street"]
        print(addr)
        df_partial = util.get_pm25(url, country, city, addr, today, AQICN_API_KEY)

        if aq_today_df is None:
            # First iteration → initialize
            aq_today_df = df_partial
        else:
            # Later iterations → append to the same DataFrame
            aq_today_df = pd.concat([aq_today_df, df_partial], ignore_index=True)


aq_today_df

tower street
salamander st
st leonards
balmwell terrace
st john's road
queensferry road


,pm25,country,city,street,date,url
0,10.0,united-kingdom,edinburgh,tower street,2025-11-18,https://api.waqi.info/feed/@11656/
1,9.0,united-kingdom,edinburgh,salamander st,2025-11-18,https://api.waqi.info/feed/@5986/
2,10.0,united-kingdom,edinburgh,st leonards,2025-11-18,https://api.waqi.info/feed/@5987/
3,6.0,united-kingdom,edinburgh,balmwell terrace,2025-11-18,https://api.waqi.info/feed/A93199/
4,11.0,united-kingdom,edinburgh,st john's road,2025-11-18,https://api.waqi.info/feed/@9580/
5,11.0,united-kingdom,edinburgh,queensferry road,2025-11-18,https://api.waqi.info/feed/@5985/


In [7]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     6 non-null      float32       
 1   country  6 non-null      object        
 2   city     6 non-null      object        
 3   street   6 non-null      object        
 4   date     6 non-null      datetime64[ns]
 5   url      6 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 392.0+ bytes


## <span style='color:#ff5f27'> 👀 Assignment for level C: Adding lagged air quality data </span>

Adding the pm25 value for previous day, two days ago and three days ago as features.

In [8]:

#Load dataframe and covert dates to not include time
features_df = air_quality_fg.read()
features_df["date"] = pd.to_datetime(features_df["date"]).dt.date

if running_c:
    #Filter out the rows corresponding to dates for yesterday, two days ago and three days ago
    yesterday_df = features_df[features_df["date"] == (today - datetime.timedelta(days=1))]
    two_days_ago_df = features_df[features_df["date"] == (today - datetime.timedelta(days=2))]
    three_days_ago = features_df[features_df["date"] == (today - datetime.timedelta(days=3))]

    #Now use previous days' values to update today's lagged values
    aq_today_df["lagged_aq_1_day"] = yesterday_df["pm25"].values[0]
    aq_today_df["lagged_aq_2_days"] = two_days_ago_df["pm25"].values[0]
    aq_today_df["lagged_aq_3_days"] = three_days_ago["pm25"].values[0]

#If running A, which sensor it is must be taken into account
if running_a:
    yesterday = today - datetime.timedelta(days=1)
    two_days_ago = today - datetime.timedelta(days=2)
    three_days_ago = today - datetime.timedelta(days=3)

    # One row per (street, date) for each lag
    yesterday_df = (features_df[features_df["date"] == yesterday][["street", "pm25"]].rename(columns={"pm25": "lagged_aq_1_day"}))

    two_days_ago_df = (features_df[features_df["date"] == two_days_ago][["street", "pm25"]].rename(columns={"pm25": "lagged_aq_2_days"}))

    three_days_ago_df = (features_df[features_df["date"] == three_days_ago][["street", "pm25"]].rename(columns={"pm25": "lagged_aq_3_days"}))

    # Merge lagged values into today's rows, per street/sensor
    aq_today_df = (aq_today_df.merge(yesterday_df, on="street", how="left").merge(two_days_ago_df, on="street", how="left").merge(three_days_ago_df, on="street", how="left"))


aq_today_df

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.75s) 


,pm25,country,city,street,date,url,lagged_aq_1_day,lagged_aq_2_days,lagged_aq_3_days
0,10.0,united-kingdom,edinburgh,tower street,2025-11-18,https://api.waqi.info/feed/@11656/,22.0,21.0,26.0
1,9.0,united-kingdom,edinburgh,salamander st,2025-11-18,https://api.waqi.info/feed/@5986/,21.0,20.0,22.0
2,10.0,united-kingdom,edinburgh,st leonards,2025-11-18,https://api.waqi.info/feed/@5987/,19.0,19.0,21.0
3,6.0,united-kingdom,edinburgh,balmwell terrace,2025-11-18,https://api.waqi.info/feed/A93199/,4.0,6.0,7.0
4,11.0,united-kingdom,edinburgh,st john's road,2025-11-18,https://api.waqi.info/feed/@9580/,18.0,20.0,22.0
5,11.0,united-kingdom,edinburgh,queensferry road,2025-11-18,https://api.waqi.info/feed/@5985/,18.0,17.0,20.0


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [9]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 56.25°N -3.0°E
Elevation 55.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-18,3.00,0.4,7.421590,50.906155,edinburgh
1,2025-11-19,2.35,0.0,17.339897,311.633453,edinburgh
2,2025-11-20,3.30,0.0,17.317459,313.315369,edinburgh
3,2025-11-21,2.35,0.0,10.195057,222.137527,edinburgh
4,2025-11-22,7.10,0.0,14.982228,234.782318,edinburgh
5,2025-11-23,5.65,0.0,5.351785,137.726379,edinburgh
6,2025-11-24,4.95,0.0,11.525623,271.789856,edinburgh


In [10]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         7 non-null      datetime64[ns]
 1   temperature_2m_mean          7 non-null      float32       
 2   precipitation_sum            7 non-null      float32       
 3   wind_speed_10m_max           7 non-null      float32       
 4   wind_direction_10m_dominant  7 non-null      float32       
 5   city                         7 non-null      object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 352.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [11]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2025-11-18 18:31:17,915 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1272015/fs/1258614/fg/1703474


Uploading Dataframe: 100.00% |██████████| Rows 6/6 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_3_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1272015/jobs/named/air_quality_3_offline_fg_materialization/executions


(Job('air_quality_3_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 747713
         }
       },
       "result": {
         "observed_value": 6.0,
         "element_count": 6,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-18T05:31:17.000914Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectati

In [12]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-11-18 18:31:32,448 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1272015/fs/1258614/fg/1703478


Uploading Dataframe: 100.00% |██████████| Rows 7/7 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: weather_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1272015/jobs/named/weather_2_offline_fg_materialization/executions
2025-11-18 18:31:48,883 INFO: Waiting for execution to finish. Current state: INITIALIZING. Final status: UNDEFINED
2025-11-18 18:31:52,176 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-18 18:33:26,875 INFO: Waiting for execution to finish. Current state: SUCCEEDING. Final status: UNDEFINED
2025-11-18 18:33:30,228 INFO: Waiting for execution to finish. Current state: FINISHED. Final status: SUCCEEDED
2025-11-18 18:33:30,777 INFO: Waiting for log aggregation to finish.
2025-11-18 18:33:30,778 INFO: Execution finished successfully.


(Job('weather_2_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 747717
         }
       },
       "result": {
         "observed_value": 5.351784706115723,
         "element_count": 7,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-18T05:31:32.000447Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": 

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
